In [8]:
library(lubridate)
library(timeDate)

In [2]:
df = read.csv('../data/seattle_911_clean.csv')
df$Timestamp <- as.POSIXct(df$Timestamp)
df$Date <- as.Date(df$Timestamp)
df$Time <- format(df$Timestamp,"%H:%M:%S")
sapply(df, typeof)

X     Address        Type    Latitude   Longitude   Timestamp 
  "integer"   "integer"   "integer"    "double"    "double"    "double" 
       Date        Time 
   "double" "character"

In [3]:
df$Weekday <- weekdays(df$Date)
df$Night <- as.integer((df$Time < "06:00:00") | (df$Time > "18:00:00"))
df$DangHoliday <- as.integer(((month(df$Date) == 7) & (day(df$Date) == 4)) |
                             ((month(df$Date) == 1) & (day(df$Date) == 1)))

In [125]:
years <- seq(2010, 2025)
tgiving <- data.frame(as.Date(USThanksgivingDay(year = years)))
colnames(tgiving) <- "dates"
xmas <- data.frame(as.Date(ChristmasDay(year = years)))
colnames(xmas) <- "dates"
trav <- rbind(tgiving, tgiving - 1, tgiving - 2, xmas, xmas - 1, xmas + 1)
df$TravHoliday <- as.integer(df$Date %in% trav$dates)

[[1]]
 [1] "2010-11-25" "2011-11-24" "2012-11-22" "2013-11-28" "2014-11-27"
 [6] "2015-11-26" "2016-11-24" "2017-11-23" "2018-11-22" "2019-11-28"
[11] "2020-11-26" "2021-11-25" "2022-11-24" "2023-11-23" "2024-11-28"
[16] "2025-11-27" "2010-11-24" "2011-11-23" "2012-11-21" "2013-11-27"
[21] "2014-11-26" "2015-11-25" "2016-11-23" "2017-11-22" "2018-11-21"
[26] "2019-11-27" "2020-11-25" "2021-11-24" "2022-11-23" "2023-11-22"
[31] "2024-11-27" "2025-11-26" "2010-11-23" "2011-11-22" "2012-11-20"
[36] "2013-11-26" "2014-11-25" "2015-11-24" "2016-11-22" "2017-11-21"
[41] "2018-11-20" "2019-11-26" "2020-11-24" "2021-11-23" "2022-11-22"
[46] "2023-11-21" "2024-11-26" "2025-11-25" "2010-12-25" "2011-12-25"
[51] "2012-12-25" "2013-12-25" "2014-12-25" "2015-12-25" "2016-12-25"
[56] "2017-12-25" "2018-12-25" "2019-12-25" "2020-12-25" "2021-12-25"
[61] "2022-12-25" "2023-12-25" "2024-12-25" "2025-12-25" "2010-12-24"
[66] "2011-12-24" "2012-12-24" "2013-12-24" "2014-12-24" "2015-12-24"
[71] "2016-12-24" "2017-12-24" "2018-12-24" "2019-12-24" "2020-12-24"
[76] "2021-12-24" "2022-12-24" "2023-12-24" "2024-12-24" "2025-12-24"
[81] "2010-12-26" "2011-12-26" "2012-12-26" "2013-12-26" "2014-12-26"
[86] "2015-12-26" "2016-12-26" "2017-12-26" "2018-12-26" "2019-12-26"
[91] "2020-12-26" "2021-12-26" "2022-12-26" "2023-12-26" "2024-12-26"
[96] "2025-12-26"

In [128]:
head(df)

X,Address,Type,Latitude,Longitude,Timestamp,Date,Time,Weekday,Night,DangHoliday,TravHoliday
1,6900 37th Av S,Medic Response,47.54068,-122.2861,2011-11-09 23:33:00,2011-11-10,23:33:00,Thursday,1,0,0
2,N 50th St / Stone Way N,Aid Response,47.66503,-122.3402,2011-11-09 23:32:00,2011-11-10,23:32:00,Thursday,1,0,0
3,E John St / E Olive Way,Aid Response,47.61957,-122.3243,2011-11-09 23:32:00,2011-11-10,23:32:00,Thursday,1,0,0
4,611 12th Av S,Aid Response,47.59741,-122.3172,2011-11-09 23:29:00,2011-11-10,23:29:00,Thursday,1,0,0
5,4545 42nd Av Sw,Automatic Medical Alarm,47.56247,-122.3855,2011-11-09 23:25:00,2011-11-10,23:25:00,Thursday,1,0,0
6,2124 3rd Av,Investigate Out Of Service,47.61335,-122.3425,2011-11-09 23:17:00,2011-11-10,23:17:00,Thursday,1,0,0
